In [1]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore, SPARQLStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
from Solr.solr import create_doc
from Jena_cli.jena import UpadateJena

DELETE ALL GRAPHS

In [2]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))

In [5]:
#Delete all Named Graph

d = """DELETE { graph ?g { ?s ?p ?o } } 

WHERE {

graph ?g {?s ?p ?o.}

}"""

store.update(d)

SOLR

In [10]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":6,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-1"}},\n  "status":"OK"}\n'

In [2]:
def UpadateSolr(path_marc, count, shelf):

    #SOLR
    #solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
    #solr.ping()
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(count)
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)
        
        count += 1
    return docs, count

In [48]:
count = 378
e = '3'
p = '5'
out = f'E{e}/P{p}'
path_marc = f'Input/{out}/koha.xml'

docs, _ = UpadateSolr(path_marc, count, f"E{e}.P{p}")

378
A conquista do êxito
379
O encontro com o eu interior
380
Palavras que levam ao sucesso
381
Modernos métodos de venda
382
Mais psicologia, melhores negócios
383
Conheça melhor a si mesmo e às pessoas com quem trata
384
O comércio através dos tempos
385
Sucesso nas relações humanas
386
Psicologia industrial
387
Probability and the logic of rational belief
388
Lógica elementar
389
Escolha e acaso
390
Lógica e existência
391
Tractatus logico-philosophicus
392
Lógica
393
Ética e saúde
394
Patentes, transgênicos e clonagem
395
As regras morais e a ética
396
Tongues of conscience
397
Capacitação para comitês de ética em pesquisa.
398
Ética social
399
Ethics, religion and, biodiversity
400
Mito e pensamento entre os gregos
401
Horizonte e complementariedade
402
Gênese, significado e ensino da filosofia no século XII
403
A filosofia contemporânea ocidental
404
As idéias de Bertrand Russell
405
Bachelard: raison et imagination = Bachelard: razão e imaginação


In [49]:
#solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":348}}\n'

IMPORT LOTE

In [ ]:
count = 1
solr_docs = list()
for e in range(1, 3):
    for p in range(1,6):
        path_marc = f'Input/E{e}/P{p}/koha.xml'
        docs, count  = UpadateSolr(path_marc, count, f"E{e}.P{p}")
        #solr_docs.append(docs)
        solr_docs = solr_docs + docs
        #count += 1
        #print(count)

In [35]:
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.add(solr_docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":1736}}\n'

JENA

In [6]:
count = 1
for i in range(1,3):
    e = i
    for p in range(1,6):
        out = f'E{e}/P{p}'
        path_marc = f'Input/{out}/koha.xml'
        shelf = f"E{e}.P{p}"
        count = UpadateJena(path_marc, count, shelf, out)


1
Conjecturas e refutações
2
Os donos da paisagem
3
Cooperação Brasil-França
4
<I=     1> Bienal de Pesquisa da Fundação Oswaldo Cruz, 7 a 11 de dezembro de 1998.
5
Contribuição da pós-graduação brasileira para o desenvolvimento sustentável
6
FAPESP
7
Para uma história da FAPESP
8
Relatório de atividades 2003-2008
9
Relatório de atividades 2007
10
Relatório de atividades 2010
11
V Jornada científica de pos-graduação da FIOCRUZ
12
Anais
13
Resumos
14
Anais
15
Resumos
16
Resumos
17
Resumos
18
Resumos
19
Congresso de iniciação científica da Universidade Federal do Amazonas
20
Resumos
21
Resumos
22
Pesquisa científica e tecnológica em saúde
23
Pesquisas recentes em energia, meio ambiente e tecnologia
24
Iniciação à pesquisa científica
25
<1º >Seminário de Ciências da Fiube
26
Encontro sobre o programa de pesquisas para o trópico úmido.
27
A pesquisa no Brasil
28
Resumos do XVII Seminário de Iniciação Científica da UFPA
29
FADESP, Fundação de Amaparo e Desenvolvimento da Pesquisa, Belém-Par

http://id.loc.gov/vocabulary/languages/    does not look like a valid URI, trying to serialize this will break.


202
Inventário e avaliação da produção técnico-científica sobre migração na Amazônia legal.


KeyError: '0'

In [16]:
e = 2
count = 106
for p in range(1,6):
    out = f'E{e}/P{p}'
    path_marc = f'Input/{out}/koha.xml'
    count = UpadateJena(path_marc, count, shelf, out)

106
An introduction to RPG-RPG II programming
107
Programacion con el lenguaje Cobol
108
Dominando o PostgreSQL
109
Study notes in system dynamics.
110
TURBO PASCAL
111
FORTH, guia do usuário
112
Princípios em projetos de programas
113
Lenguajes de diagramas de flujo
114
Linguagem PASCAL
115
A simplified guide to FORTRAN programming
116
Lingugem C
117
Biblioteca do programador Clipper 5.01
118
Delphi 2
119
Manipulando bancos de dados com Delphi 2
120
SQL para Delphi 2
121
Cobol y sus aplicaciones en los negocios
122
Clipper
123
Pascal e técnicas de programação
124
I simpósio brasileiro de linguagens de programação
125
MUMPS: uma nova abordagem
126
turbo Pascal
127
Supercalc
128
Fortran
129
Fortran - Monitor
130
Quattro Pro 2
131
Desktop publishing com pagemaker IBM PC AT, PS.2 e compatíveis
132
Macro processors and techniques for portable software
133
TURBO BASIC
134
Princípios de sistemas de gerência de bancos de dados distribuídos
135
Sistemas de gerência de bancos de dados distribuí

http://id.loc.gov/vocabulary/languages/0 p does not look like a valid URI, trying to serialize this will break.


141
Dbase III plus


Exception: "http://id.loc.gov/vocabulary/languages/0 p" does not look like a valid URI, I cannot serialize this as N3/Turtle. Perhaps you wanted to urlencode it?

In [15]:
count

106